# Multi-Model Chatbot using Gradio

A conversational chatbot with streaming output and model selection (GPT-4.1-mini / Claude Sonnet 4.5).

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)

openai_client = OpenAI()
anthropic_client = OpenAI(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    base_url="https://api.anthropic.com/v1/",
)

In [ ]:
system_message = "You are a helpful assistant"

MODELS = {
    "GPT-4.1-mini": ("gpt-4.1-mini", openai_client),
    "Claude Sonnet 4.5": ("claude-sonnet-4-5-20250929", anthropic_client),
}

In [ ]:
def chat(message, history, model_name):
    model_id, client = MODELS[model_name]
    messages = [{"role": "system", "content": system_message}]
    for h in history:
        messages.append({"role": h["role"], "content": h["content"]})
    messages.append({"role": "user", "content": message})

    stream = client.chat.completions.create(model=model_id, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [ ]:
demo = gr.ChatInterface(
    fn=chat,
    type="messages",
    additional_inputs=[
        gr.Dropdown(list(MODELS.keys()), label="Select model", value="GPT-4.1-mini"),
    ],
    title="Multi-Model Chatbot",
)
demo.launch()